In [1]:
# imports
import os
import math
import tensorflow as tf
from tensorflow import keras
import numpy as np
from numpy import genfromtxt
from numpy.lib import recfunctions as rfn
#print(tf.__version__)

In [2]:
# data
dtypes = {
    'fraud_bool': '?',
    'income': 'f4',
    'name_email_similarity': 'f2',
    'prev_address_months_count': 'i2',
    'current_address_months_count': 'i2',
    'customer_age': 'u1',
    'days_since_request': 'f2',
    'intended_balcon_amount': 'f2',
    'payment_type': 'u1', # STR
    'zip_count_4w': 'u2',
    'velocity_6h': 'f4',
    'velocity_24h': 'f4',
    'velocity_4w': 'f4',
    'bank_branch_count_8w': 'u2',
    'date_of_birth_distinct_emails_4w': 'u1',
    'employment_status': 'u2',
    'credit_risk_score': 'i2',
    'email_is_free': 'u1',
    'housing_status': 'u1', # STR
    'phone_home_valid': 'u1',
    'phone_mobile_valid': 'u1',
    'bank_months_count': 'i1',
    'has_other_cards': 'u1',
    'proposed_credit_limit':
    'f2', 'foreign_request': 'u1',
    'source': 'u1', # STR
    'session_length_in_minutes': 'f4',
    'device_os': 'u1', # STR
    'keep_alive_session': 'u1',
    'device_distinct_emails_8w': 'i1',
    'device_fraud_count': 'u1',
    'month': 'u1'}

def strconv(cat):
    """Returns a function that maps its input string to the same small number each time."""
    seen = {}
    def inner(s):
        nonlocal seen
        if s not in seen:
            seen[s] = len(seen)
        return seen[s]
    return inner

strconvs = {
    'source': strconv("source"),
    'device_os': strconv("device_os"),
    'housing_status': strconv("housing_status"),
    'payment_type': strconv("payment_type"),
    'employment_status': strconv("employment_status")
}

column_labels = list(next(open('Base.csv')).strip().split(','))
converters = {0: lambda s: bool(int(s))}
for k, v in strconvs.items():
    converters[column_labels.index(k)] = v
rawdata = genfromtxt('Base.csv', dtype=[dtypes[n] for n in column_labels],
                  names=column_labels, delimiter=',', converters=converters)[1:] # skip header
txtdata = list(open('Base.csv').read().split('\n'))[1:]
print(f'Example row: {rawdata[0]} : {type(rawdata[0])}')
print(f'Computed type of elements: {rawdata.dtype}')
print(f'Same row but unparsed, does it match? {txtdata[0]})')
issue = False
anyissue = False
for rowix, (txtrow, rawrow) in enumerate(zip(txtdata, rawdata)):
    if not (rawrow[0] == 0 or rawrow[0] == 1):
        print(f'WARN: fraud_bool for row {rowix} is {rawrow[0]}, not 0 or 1!')
        anyissue = True

    for ix, c in enumerate(rawrow):
        if ix == 4 and c == 65535:
            continue # this is the one column where -1 as missing is OK
        if type(c) != np.str_ and np.isnan(c):
            print(f'{rawrow} has a nan at {ix}!')
        elif np.issubdtype(type(c), np.integer):
            info = np.iinfo(type(c))
            if (c == info.min or c == info.max) and c!=0:
                issue = True
                print(f'WARN: column {ix} row {rowix} ({column_labels[ix]}) has maxed out the irange of {type(c)}!')
        elif np.issubdtype(type(c), np.floating):
            info = np.finfo(type(c))
            if c == info.min or c == info.max:
                issue = True
                print(f'WARN: column {ix} row {rowix} ({column_labels[ix]}) has maxed out the frange of {type(c)}!')
    if issue:
        print(f'row was: {rawrow}')
        print(f'source data was: {txtrow}')
        anyissue = True
        issue = False
else:
    if not anyissue:
        print('No issues found with datatype!')
    else:
        print('Issues found with datatype! Look closely!')
data = rfn.structured_to_unstructured(rawdata, dtype=np.float32)

Example row: (True, 0.9, 0.1669, -1, 88, 50, 0.02092, -1.331, 1, 769, 10650.766, 3134.3196, 3863.6477, 1, 6, 1, 185, 0, 1, 1, 0, 24, 0, 500., 0, 1, 3.8881147, 1, 0, 1, 0, 7) : <class 'numpy.void'>
Computed type of elements: [('fraud_bool', '?'), ('income', '<f4'), ('name_email_similarity', '<f2'), ('prev_address_months_count', '<i2'), ('current_address_months_count', '<i2'), ('customer_age', 'u1'), ('days_since_request', '<f2'), ('intended_balcon_amount', '<f2'), ('payment_type', 'u1'), ('zip_count_4w', '<u2'), ('velocity_6h', '<f4'), ('velocity_24h', '<f4'), ('velocity_4w', '<f4'), ('bank_branch_count_8w', '<u2'), ('date_of_birth_distinct_emails_4w', 'u1'), ('employment_status', '<u2'), ('credit_risk_score', '<i2'), ('email_is_free', 'u1'), ('housing_status', 'u1'), ('phone_home_valid', 'u1'), ('phone_mobile_valid', 'u1'), ('bank_months_count', 'i1'), ('has_other_cards', 'u1'), ('proposed_credit_limit', '<f2'), ('foreign_request', 'u1'), ('source', 'u1'), ('session_length_in_minutes',

Now that we've parsed the data and verified some basic properties about it, we are ready to further process it into a useful form. First, we will min-max normalize the data:

In [3]:
# min-max normalize each column

# TODO

In [4]:
is_fraud_ground_truth = np.int_(data[:,:1])
real_dataset = data[:,1:]  # skip the fraud bool
#np.array([np.array(row)[1:] for row in rawdata]) # skip the fraud bool
training_size = math.floor(len(real_dataset)*.2)
training_dataset = real_dataset[:training_size]
evaluation_dataset = real_dataset[training_size:]
evaluation_dataset_labels = is_fraud_ground_truth[training_size:]
is_fraud_ground_truth_training = is_fraud_ground_truth[:training_size]


In [5]:
def dbg_ndarray(val):
    print(type(val))
    print(type(val.dtype))
    print(val.shape)
dbg_ndarray(is_fraud_ground_truth)
dbg_ndarray(is_fraud_ground_truth_training)
print(training_dataset[3:10])
dbg_ndarray(training_dataset)

<class 'numpy.ndarray'>
<class 'numpy.dtype[int64]'>
(1000000, 1)
<class 'numpy.ndarray'>
<class 'numpy.dtype[int64]'>
(200000, 1)
[[ 8.9999998e-01  1.5954590e-01 -1.0000000e+00  2.2000000e+01
   5.0000000e+01  1.9073486e-02 -1.2050781e+00  2.0000000e+00
   8.1000000e+02  3.4570640e+03  4.0549084e+03  3.0222617e+03
   1.9210000e+03  6.0000000e+00  1.0000000e+00  1.1000000e+02
   1.0000000e+00  1.0000000e+00  0.0000000e+00  1.0000000e+00
   3.1000000e+01  1.0000000e+00  2.0000000e+02  0.0000000e+00
   1.0000000e+00  2.0479045e+00  3.0000000e+00  0.0000000e+00
   1.0000000e+00  0.0000000e+00  7.0000000e+00]
 [ 8.9999998e-01  5.9619141e-01 -1.0000000e+00  2.1800000e+02
   5.0000000e+01  4.4403076e-03 -7.7343750e-01  2.0000000e+00
   8.9000000e+02  5.0203418e+03  2.7282371e+03  3.0876709e+03
   1.9900000e+03  2.0000000e+00  1.0000000e+00  2.9500000e+02
   1.0000000e+00  1.0000000e+00  1.0000000e+00  0.0000000e+00
   3.1000000e+01  0.0000000e+00  1.5000000e+03  0.0000000e+00
   1.0000000e+0

In [6]:
print(real_dataset[0:2])

[[ 8.9999998e-01  1.6687012e-01 -1.0000000e+00  8.8000000e+01
   5.0000000e+01  2.0919800e-02 -1.3310547e+00  1.0000000e+00
   7.6900000e+02  1.0650766e+04  3.1343196e+03  3.8636477e+03
   1.0000000e+00  6.0000000e+00  1.0000000e+00  1.8500000e+02
   0.0000000e+00  1.0000000e+00  1.0000000e+00  0.0000000e+00
   2.4000000e+01  0.0000000e+00  5.0000000e+02  0.0000000e+00
   1.0000000e+00  3.8881147e+00  1.0000000e+00  0.0000000e+00
   1.0000000e+00  0.0000000e+00  7.0000000e+00]
 [ 8.9999998e-01  2.9638672e-01 -1.0000000e+00  1.4400000e+02
   5.0000000e+01  5.4168701e-03 -8.1640625e-01  2.0000000e+00
   3.6600000e+02  5.3404730e+02  2.6709182e+03  3.1242981e+03
   7.1800000e+02  3.0000000e+00  1.0000000e+00  2.5900000e+02
   1.0000000e+00  1.0000000e+00  0.0000000e+00  0.0000000e+00
   1.5000000e+01  0.0000000e+00  1.5000000e+03  0.0000000e+00
   1.0000000e+00  3.1798819e+01  1.0000000e+00  0.0000000e+00
   1.0000000e+00  0.0000000e+00  7.0000000e+00]]


In [7]:
#Model old
model: keras.Model = keras.Sequential([
    keras.layers.Dense((32**2) * 4, activation='relu'),
    keras.layers.Dense((32**2) * 2, activation='relu'),
    keras.layers.Dense(32**2, activation='relu'),
    keras.layers.Dense((32**2)/4, activation='relu'),

    keras.layers.Dense(1)
])

Metal device set to: Apple M1

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2022-12-11 09:58:40.797842: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-12-11 09:58:40.798015: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [8]:
#compile
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])


In [9]:
#fit
history = model.fit(x=training_dataset, y=is_fraud_ground_truth_training, epochs=3, validation_split=0.1)

Epoch 1/3


2022-12-11 09:58:40.946215: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-12-11 09:58:41.298225: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


5623/5625 [============================>.] - ETA: 0s - loss: 0.2436 - accuracy: 0.9842

2022-12-11 10:00:19.100127: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


5625/5625 [==============================] - 100s 18ms/step - loss: 0.2436 - accuracy: 0.9842 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 2/3
1136/5625 [=====>........................] - ETA: 1:20 - loss: 0.2440 - accuracy: 0.9842

In [ ]:
history = model.evaluate(evaluation_dataset, evaluation_dataset_labels)

15749/25000 [=================>............] - ETA: 36s - loss: 0.1636 - accuracy: 0.9894

KeyboardInterrupt: 

In [ ]:
model.predict(evaluation_dataset[5])